# Commander.js 实践教程

本教程配合 `demo.js` 文件使用，演示 Commander.js 的核心概念。

⚠️ **重要提示**：
- 本教程使用 **Deno 内核**，代码块中的 `require` 语法**不能直接运行**
- 所有可执行代码都在 `demo.js` 中，使用 **Node.js** 运行
- 本教程用于概念讲解，实际运行请在终端执行 `node demo.js`

## 1. 概念：选项定义与获取

### 定义选项

```javascript
// demo.js 中的代码
program
  .option('-p, --port <number>', '服务端口号', '3000')  // 带值选项
  .option('-v, --verbose', '显示详细日志')               // 布尔选项
  .option('-f, --force', '强制执行');                   // 布尔选项
```

### 映射关系

| 定义中的长格式 | 生成的属性名 | 值类型 |
|---------------|-------------|--------|
| `--port` | `options.port` | string |
| `--verbose` | `options.verbose` | boolean |
| `--force` | `options.force` | boolean |

### 在 action 中获取选项

```javascript
.action(function() {
  // 获取当前命令的选项
  const cmdOptions = this.opts();
  
  // 获取全局选项
  const globalOptions = program.opts();
  
  console.log(cmdOptions.port);     // '8080'
  console.log(globalOptions.verbose); // true
});
```

## 2. 实践：运行 demo.js

在终端中执行以下命令：

### 2.1 查看帮助
```bash
node demo.js --help
```

输出包含：
- 全局选项：`-v, --verbose`, `-c, --config`
- 子命令：`start`, `stop`, `list`, `config`

### 2.2 测试 start 命令
```bash
# 使用默认值
node demo.js start

# 自定义端口
node demo.js start -p 8080

# 同时使用全局选项和命令选项
node demo.js start -p 9000 --host 0.0.0.0 -v -f
```

### 2.3 观察输出
运行上面的命令后，你会看到：
```
🚀 执行 start 命令
─────────────────────────
全局选项:
  verbose: true
  config:  ./config.json
命令选项:
  port:    9000
  host:    0.0.0.0
  force:   true
```

**重点理解**：
- `-v`（全局选项）出现在 `start` 之前还是之后效果一样
- `-p 9000` 是 start 命令的专属选项
- 默认值 `'3000'` 在用户未提供 `-p` 时生效

## 3. 概念：子命令

### 子命令定义

```javascript
// 定义子命令
program
  .command('start')                    // 命令名
  .description('启动服务')              // 描述
  .option('-p, --port <number>', '', '3000')  // 专属选项
  .action(function() {                  // 执行逻辑
    const opts = this.opts();
    console.log('端口:', opts.port);
  });
```

### 子命令 vs 全局

| 特性 | 全局 program | 子 command |
|------|-------------|-----------|
| 获取方式 | `program.opts()` | `this.opts()` |
| 作用范围 | 所有命令 | 仅当前命令 |
| 定义位置 | `program.option()` | `command.option()` |

## 4. 实践：测试 list 命令（带参数）

`list` 命令演示了**位置参数**的使用：

```javascript
// 定义：<> 表示必填参数
.command('list <pattern>')

// 在 action 中接收参数
.action(function(pattern) {
  console.log(pattern);  // '*.js'
});
```

### 运行测试
```bash
# 基本用法
node demo.js list '*.js'

# 带选项
node demo.js list '*.js' -l

# 多选项组合
node demo.js list '*' -la --sort size
```

### 输出解析
```
📋 执行 list 命令
─────────────────────────
全局选项:
  verbose: false
参数:
  pattern: *.js     ← 位置参数
命令选项:
  long:    true     ← -l
  all:     true     ← -a
  sort:    size     ← --sort size
```

## 5. 核心知识点总结

### 选项定义的完整语法

```javascript
.option('-短格式, --长格式 <值类型>', '描述', 默认值)
//      ↑↑        ↑↑↑↑    ↑↑↑↑↑↑                  ↑↑↑↑
//      快捷方式   属性名   占位符（决定是否需要值）   可选
```

### 属性名映射规则

| 定义代码 | 属性名 | 命令行使用 |
|----------|--------|-----------|
| `-p, --port <n>` | `options.port` | `--port 8080` 或 `-p 8080` |
| `-v, --verbose` | `options.verbose` | `--verbose` 或 `-v` |
| `--host <s>` | `options.host` | `--host 0.0.0.0` |

### 获取选项的两种方式

```javascript
// 方式 1：子命令 action 中
.action(function() {
  const opts = this.opts();  // 当前命令的选项
  const global = program.opts();  // 全局选项
});

// 方式 2：顶级命令（没有子命令时）
program.parse(process.argv);
const options = program.opts();  // 所有选项
```

## 6. 练习任务

尝试修改 `demo.js`，完成以下任务：

### 练习 1：添加新选项
给 `start` 命令添加 `--env` 选项，支持 `development` 和 `production`：
```bash
node demo.js start --env production
# 输出: env: production
```

### 练习 2：添加新命令
添加 `status` 命令，显示服务状态：
```bash
node demo.js status
# 输出: 服务状态: running
```

### 练习 3：默认值测试
测试各种情况下选项的值：
```bash
# 不提供任何选项，观察默认值
node demo.js start

# 只提供部分选项
node demo.js start -p 8080

# 提供所有选项
node demo.js start -p 8080 --host 0.0.0.0 -f -v
```

## 7. 与 OpenClaw 的关联

OpenClaw 使用 Commander.js 的方式：

```javascript
// OpenClaw 的简化版结构
const program = new Command();

// 全局选项
program.option('--verbose', '详细输出');

// gateway 子命令组
const gateway = program.command('gateway');

// gateway run
gateway
  .command('run')
  .option('-p, --port <number>', '端口', 18789)
  .option('--bind <mode>', '绑定模式')
  .action(async function() {
    const opts = this.opts();
    await startGateway(opts.port, opts.bind);
  });

// 实际使用
// openclaw gateway run --port 8080
```

你现在理解的 Commander.js 概念，可以直接应用到 OpenClaw 的 CLI 代码阅读中！